https://towardsdatascience.com/how-to-train-a-bert-model-from-scratch-72cfce554fc6

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("reuters21578", "ModHayes")

In [ ]:
dataset['train'].features

In [ ]:
from tqdm.auto import tqdm

text_data = []
file_count = 0

for sample in tqdm(dataset['train']):
  sample = sample['text'].replace('\n', '')
  text_data.append(sample)
  if len(text_data) == 10_000:
    with open(f'reuters_data/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
      fp.write('\n'.join(text_data))
    text_data = []
    file_count += 1

# after saving in 10K chunks, we will have ~2082 leftover samples, we save those now too
with open(f'reuters_data/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
  fp.write('\n'.join(text_data))

In [ ]:
from pathlib import Path
paths = [str(x) for x in Path('reuters_data').glob('**/*.txt')]

In [7]:
from tokenizers import ByteLevelBPETokenizer

tokenizer = ByteLevelBPETokenizer()
tokenizer.train(files=paths[:5], vocab_size=30_522, min_frequency=2,
                special_tokens=['<s>', '<pad>', '</s>', '<unk>', '<mask>'])

NameError: name 'paths' is not defined

In [ ]:
import os

# os.mkdir('./reuterbert')

tokenizer.save_model('reuterbert')

In [8]:
from transformers import RobertaTokenizer

# initialize the tokenizer using the tokenizer we initialized and saved to file
tokenizer = RobertaTokenizer.from_pretrained('reuterbert', max_len=512)

file reuterbert/config.json not found
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'RobertaTokenizer'.


In [9]:
tokens = tokenizer('hello, friend?')
tokens.input_ids

[0, 2908, 1054, 16, 10620, 35, 2]

In [10]:
lines = None
with open('reuters_data/text_0.txt', 'r', encoding='utf-8') as fp:
  lines = fp.read().split('\n')
batch = tokenizer(lines, max_length=512, padding='max_length', truncation=True)
# batch = batch['input_ids']

In [11]:
import torch

labels = torch.tensor([x for x in batch['input_ids']])
mask = torch.tensor([x for x in batch['attention_mask']])

In [12]:
# make copy of labels tensor, this will be input_ids
input_ids = labels.detach().clone()
# create random array of floats with equal dims to input_ids
rand = torch.rand(input_ids.shape)
# mask random 15% where token is not 0 [PAD], 1 [CLS], or 2 [SEP]
mask_arr = (rand < .15) * (input_ids != 0) * (input_ids != 1) * (input_ids != 2)
# loop through each row in input_ids tensor (cannot do in parallel)
for i in range(input_ids.shape[0]):
    # get indices of mask positions from mask array
    selection = torch.flatten(mask_arr[i].nonzero()).tolist()
    # mask input_ids
    input_ids[i, selection] = 3  # our custom [MASK] token == 3

In [13]:
input_ids.shape

torch.Size([10000, 512])

In [14]:
encodings = {'input_ids': input_ids, 'attention_mask': mask, 'labels': labels}

In [15]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        # store encodings internally
        self.encodings = encodings

    def __len__(self):
        # return the number of samples
        return self.encodings['input_ids'].shape[0]

    def __getitem__(self, i):
        # return dictionary of input_ids, attention_mask, and labels for index i
        return {key: tensor[i] for key, tensor in self.encodings.items()}

In [16]:
dataset = Dataset(encodings)

In [17]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [18]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=30_522,  # we align this to the tokenizer vocab_size
    max_position_embeddings=514,
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1
)

In [19]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config)

In [20]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

In [21]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=1e-4)

In [23]:
from tqdm.auto import tqdm

epochs = 2

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

Epoch 0:  98%|█████████▊| 613/625 [7:35:20<08:15, 41.32s/it, loss=0.555]

In [ ]:
model.save_pretrained('./reuterbert')  # and don't forget to save reuterBERT!